<a href="https://colab.research.google.com/github/maimuna-noshin/Solar_Prediction/blob/main/solarprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [345]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, BatchNormalization, Flatten
from sklearn.preprocessing import StandardScaler
from scipy.fftpack import fft
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [346]:
df=pd.read_csv("/content/dataset_modified1.csv")

In [347]:
# Convert to datetime
df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])




In [348]:
# Selecting features (X) and target variables (Y)
features = ['Hour', 'Temperature_A','Pressure_A', 'Day','Month', 'Surface Albedo_A', 'Cloud Type_A']
targets = ['DNI_A']  # Target variables for all three locations

In [349]:

X = df[features].values
y = df[targets].values

In [350]:
# Keep timestamps before scaling
timestamps = df[['Month', 'Day', 'Hour']]

In [351]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler(feature_range=(0, 1000))
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
# Save the feature scaler
with open("scaler_X.pkl", "wb") as f:
    pickle.dump(scaler_X, f)

# Save the target scaler
with open("scaler_y.pkl", "wb") as f:
    pickle.dump(scaler_y, f)

In [352]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test, timestamps_train, timestamps_test = train_test_split(X_scaled, y_scaled,timestamps, test_size=0.2, random_state=42)
print(X_train.max())

1.0


In [353]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(targets), activation='linear')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [354]:
# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [355]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

Epoch 1/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 71814.0000 - mae: 200.3396 - val_loss: 36198.6172 - val_mae: 138.5720
Epoch 2/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 38147.0742 - mae: 129.7633 - val_loss: 30625.5508 - val_mae: 108.1315
Epoch 3/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 32921.7773 - mae: 111.8164 - val_loss: 28487.9902 - val_mae: 102.4299
Epoch 4/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 32482.8691 - mae: 109.0581 - val_loss: 28705.9590 - val_mae: 100.8421
Epoch 5/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 29869.7520 - mae: 103.9493 - val_loss: 28101.9648 - val_mae: 98.7062
Epoch 6/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 29358.0430 - mae: 102.0701 - val_loss: 28166.8555 - val_mae: 94.6625
Epoch 7/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 28029.4785 - mae: 99.0453 - val_loss: 27491.3926 - val_mae: 94.3444
Epoch 8/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 28426.3359 - mae: 97.6579 - val_lo

In [356]:
# Predict future solar irradiance for all three locations
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
# Set negative irradiance values to 0
y_pred[y_pred < 10] = 0


55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [357]:
# Convert predictions into a DataFrame
predictions_df = timestamps_test.copy()

# Assign each column separately
predictions_df[['DNI_A']] = y_pred

In [358]:
# Print first 100 predictions with date and hour
print(predictions_df.head(100))
predictions_df.to_csv("solar_irradiance_predictions.csv", index=False)  # Saves to a CSV]

      Month  Day  Hour       DNI_A
6056      9   10     8  189.921402
5556      8   20    12  295.104919
5990      9    7    14  298.992828
7674     11   16    18   35.227814
3319      5   19     7   92.483604
...     ...  ...   ...         ...
6006      9    8     6   11.654873
8063     12    2    23    0.000000
14        1    1    14  441.637817
5306      8   10     2    0.000000
8524     12   22     4    0.000000

[100 rows x 4 columns]


In [359]:
# Save using the new format:
model.save("solar_irradiance_predictions.keras")  # Or model.save("my_model") - the .keras is optional

# Load using the new format:
model = keras.models.load_model("solar_irradiance_predictions.keras")  # Or model = keras.models.load_model("my_model")

In [360]:
#  Load the scalers used during training

with open("scaler_X.pkl", "rb") as f:
    scaler_X = pickle.load(f)
with open("scaler_y.pkl", "rb") as f:
    scaler_y = pickle.load(f)

In [361]:
# Generate timestamps for the first week of March 2025 (hourly)
date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")
future_df = pd.DataFrame({"datetime": date_range})

<ipython-input-361-0e223ba96d09>:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")


In [362]:
#  Feature Engineering
future_df["day"] = future_df["datetime"].dt.day
future_df["month"] = future_df["datetime"].dt.month
future_df["hour"] = future_df["datetime"].dt.hour
future_df.drop(columns=["datetime"], inplace=True)

In [363]:
# Generate random values
future_df["Temperature_A"] = np.random.uniform(27, 30, size=168)  # Temperature between 15°C to 30°C
future_df["Pressure_A"] = np.random.uniform(1000, 1050, size=168)  # Pressure between 950-1050 hPa
future_df["Surface Albedo_A"] = np.random.uniform(0.1, 0.5, size=168)  # Albedo between 0.1 - 0.5
future_df["Cloud Type_A"] = np.random.randint(0, 4, size=168)  # Cloud types (0-3)

# Merge the first week’s random values with `future_df`
#future_df = future_df.merge(future_df, on=["Month", "Day"], how="left")

# Time-based encoding (sin/cos transformation)
#future_df["hour_sin"] = np.sin(2 * np.pi * future_df["hour"] / 24)
#future_df["hour_cos"] = np.cos(2 * np.pi * future_df["hour"] / 24)

# Select the same features used in training
#X_future = future_df[['hour_sin', 'hour_cos', 'Temperature_A', 'Pressure_A', 'day', 'Month', 'Surface Albedo_A', 'Cloud Type_A']].values

In [364]:
#  Select the same features used in training
X_future = future_df[['hour', 'Temperature_A','Pressure_A', 'day','month', 'Surface Albedo_A', 'Cloud Type_A']].values
#  Scale future data using the trained scaler
X_future_scaled = scaler_X.transform(X_future)
print(X_future_scaled)


[[0.         0.66663715 1.30479688 ... 0.18181818 0.12693279 0.        ]
 [0.04347826 0.62255066 1.45797695 ... 0.18181818 2.2174574  0.33333333]
 [0.08695652 0.59606498 1.56298819 ... 0.18181818 2.61932377 0.33333333]
 ...
 [0.91304348 0.64178887 0.34258952 ... 0.18181818 3.76493378 0.        ]
 [0.95652174 0.60998111 1.53510776 ... 0.18181818 4.89006272 0.11111111]
 [1.         0.64507398 1.88683215 ... 0.18181818 1.76131616 0.33333333]]


In [365]:
#  Make predictions
y_future_scaled = model.predict(X_future_scaled)

#  Inverse scale predictions
y_future = scaler_y.inverse_transform(y_future_scaled)
#  Set negative irradiance values to 0
y_future[y_future < 15] = 0

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [366]:
# Save predictions to DataFrame
future_df[['DNI_A']] = y_future

#  Save to CSV
future_df.to_csv("Future_Solar_Predictions_March2025.csv", index=False)

#Display first 10 predictions
print(future_df.head(10))

   day  month  hour  Temperature_A   Pressure_A  Surface Albedo_A  \
0    1      3     0      29.932371  1030.143906          0.146347   
1    1      3     1      28.495152  1034.739309          0.250873   
2    1      3     2      27.631718  1037.889646          0.270966   
3    1      3     3      28.907895  1014.859681          0.481431   
4    1      3     4      28.095491  1022.325142          0.108858   
5    1      3     5      29.233576  1014.762336          0.464029   
6    1      3     6      28.488764  1003.650505          0.478334   
7    1      3     7      28.439515  1015.293920          0.411585   
8    1      3     8      29.735815  1006.691396          0.216200   
9    1      3     9      29.638777  1016.481571          0.170367   

   Cloud Type_A       DNI_A  
0             0    0.000000  
1             3    0.000000  
2             3    0.000000  
3             2    0.000000  
4             0    0.000000  
5             1    0.000000  
6             1    0.000000  
